In [2]:
import pandas as pd
import re

In [3]:
# load the dataset from HF
df_full = pd.read_parquet("hf://datasets/ruggsea/stanford-encyclopedia-of-philosophy_chat_multi_turn/data/train-00000-of-00001.parquet")

C:\Python_files\zipf-law-for-information-theory\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [18]:
# I recommend to work on mock dataset (50 first rows) for writing the code
df = df_full.head(20)
df.head()

,conversation,prompt
0,"[{'content': 'Professor, I was thinking about ...",You are an expert and well-read Philosophy pro...
1,"[{'content': 'Professor Phil, do we always cho...",You are an expert and well-read Philosophy pro...
2,"[{'content': 'Professor Phil, I was reading th...",You are an expert and well-read Philosophy pro...
3,"[{'content': 'Professor Phil, I've been wonder...",You are an expert and well-read Philosophy pro...
4,"[{'content': 'Professor, I've been thinking ab...",You are an expert and well-read Philosophy pro...


In [19]:
def extract_content(conversation: dict):
    """
    Unwraps original dictionary with student's and professor utterances.
    :param conversation: dict with utterances
    :return: list of professor utterances and student utterances
    """
    professor = []
    user = []
    for turn in conversation:
        # print(turn)
        # print('-'*20)
        if turn['role'] == 'user':
            user.append(turn['content'])
        else:
            professor.append(turn['content'])
    return professor, user

def prepare_prompt(prompt: str):
    """
    Cleans the redundant content from each prompt. Designed to use with lambda.
    :param prompt: prompt to clean
    :return: cleaned prompt
    """
    return re.findall(r'\"([\s\S]+?)\"', prompt)


In [20]:
# Split the conversation into various rows in df
# select utterances of professor
df['professor'] = df['conversation'].apply(lambda x: extract_content(x)[0])

# select utterances of student
df['student'] = df['conversation'].apply(lambda x: extract_content(x)[1])

# clean redundant content from prompt
df['prompt'] = df['prompt'].apply(lambda x: prepare_prompt(x))

C:\Users\quiz0\AppData\Local\Temp\ipykernel_26892\1562085957.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['professor'] = df['conversation'].apply(lambda x: extract_content(x)[0])
C:\Users\quiz0\AppData\Local\Temp\ipykernel_26892\1562085957.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['student'] = df['conversation'].apply(lambda x: extract_content(x)[1])
C:\Users\quiz0\AppData\Local\Temp\ipykernel_26892\1562085957.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a s

In [21]:
df = df.drop('conversation', axis=1)

In [22]:
df.head()

,prompt,professor,student
0,[You happen to know that Tim and Harry have re...,"[That's a great question! You see, our brains ...","[Professor, I was thinking about how I conclud..."
1,[\n Even if it is true that we routinely rely ...,"[Well, my inquisitive student, that's a great ...","[Professor Phil, do we always choose the simpl..."
2,[\n The Buddha’s discourses collected in the Ā...,"[Ah, you're onto something. Think of a movie. ...","[Professor Phil, I was reading this book about..."
3,"[What is an ability? On one reading, this ques...",[That's a great question. Think of it like thi...,"[Professor Phil, I've been wondering, how do w..."
4,[\n It will be helpful to begin by considering...,[That's an intriguing question. To explore thi...,"[Professor, I've been thinking about abilities..."


In [23]:
df = df.explode(["professor", "student"]) # powerful line! creates a pair of student-professor utterance for each row
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 83 entries, 0 to 19
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   prompt     83 non-null     object
 1   professor  83 non-null     object
 2   student    83 non-null     object
dtypes: object(3)
memory usage: 2.6+ KB


In [24]:
# explode made unwrapped columns 'professor' and 'student' to strings. For consitency we have to do the sam with 'prompt'
df['prompt'] = df['prompt'].apply(lambda x: ' '.join(map(str, x)))

In [25]:
df.head()

,prompt,professor,student
0,You happen to know that Tim and Harry have rec...,"That's a great question! You see, our brains a...","Professor, I was thinking about how I conclude..."
0,You happen to know that Tim and Harry have rec...,"Of course, you could be wrong! But here's the ...","But isn't that just an assumption? I mean, I c..."
0,You happen to know that Tim and Harry have rec...,"Ah, great example! This is where the concept o...","That makes sense, I guess. But what about case..."
0,You happen to know that Tim and Harry have rec...,Exactly! Our minds are wired to seek patterns ...,"Yeah, I think so. It's like, our brains are al..."
1,\n Even if it is true that we routinely rely o...,"Well, my inquisitive student, that's a great q...","Professor Phil, do we always choose the simple..."


In [26]:
# Now data is clean
df.to_pickle('dfs/preprocessed-df.pkl') # save it for future work